## Herso 1 sailing alone
### Estimate the power-speed relationship

### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import itertools
# Used for mathematical functions
import math             
import logging

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
# import pyproj
# import shapely.geometry
# from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt
import tqdm


import plotly.express as px
from plotly.subplots import make_subplots

# OpenTNSim
import opentnsim

# turn on debug messages if want to see what's going on in detail
# logging.basicConfig(level=logging.DEBUG)
# turn off debug messages, only show info and higher level messages
logging.basicConfig(level=logging.INFO)

### Create vessel - add VesselProperties and ConsumesEnergy mixins

In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# Create a dict with all important settings

data_vessel = {
    "env": None,
    "name": 'Vessel M6',
    "route": None,
    "geometry": None,
    "v": None,  # m/s
    "V_g_ave":3,
    "use_V_g_profile":False,
    "type": None,
    "B": 9.5,
    "L": 85,
    "H_e": None, 
    "H_f": None, 
    "T": 2,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": False, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Dry_DH", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "P_installed": 780.0,   
    "P_tot_given": None, # kW
    "P_tot_given_profile":False,
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": False,
    "sailing_upstream":False,
    "wind_influence": False,
    "P_hotel_perc": 0.064,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 2.0 ,
    "C_B":0.85, 
    "C_year": 1961,
}             



In [4]:
data_vessel_i = data_vessel.copy()
vessel = TransportResource(**data_vessel_i)

### Set vessel properties and sailing conditions ( input value )

In [5]:
V_s = np.linspace(0.1, 7, 801)          # ship sailing speeds to water, (m/s)
# V_s_km_h =  np.linspace(3,20,35)

h_0 = [3,3.5,4,5,6,7,8,9,10,20]                 # water depths,(m)
T = [2]   # draught


### prepare input matrix for calculation

In [6]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(T, h_0, V_s))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"T": item[0], "h_0": item[1], "V_s": item[2]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.tail()

,T,h_0,V_s
8005,2,20.0,6.965500
8006,2,20.0,6.974125
8007,2,20.0,6.982750
8008,2,20.0,6.991375
8009,2,20.0,7.000000


### Run OpenTNSim to Calculate resistance, required power and emissions 

In [7]:

results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    
    data_vessel_i = data_vessel.copy()
    
    vessel = TransportResource(**data_vessel_i)
    vessel._T = row['T']
    V_s = row['V_s']
    h_0 = row['h_0']
    vessel.calculate_properties() # L is used here in the computation of L_R
    # h_0 = vessel.calculate_h_squat(v = V_s, h_0 = h_0)
    # print(h_0)
    R_f = vessel.calculate_frictional_resistance(V_s, h_0) 
    R_f_one_k1 = vessel.calculate_viscous_resistance()
    R_APP = vessel.calculate_appendage_resistance(V_s, h_0)
    R_W = vessel.calculate_wave_resistance(V_s, h_0)
    R_res = vessel.calculate_residual_resistance(V_s, h_0)
    R_T = vessel.calculate_total_resistance(V_s, h_0)
    (eta_D, P_d,P_propulsion,P_tot,P_given) = vessel.calculate_total_power_required(V_s, h_0)

    
    result = {}
    result.update(row)
    result['P_installed'] = vessel.P_installed
    result['R_f'] = R_f
    result['R_f_one_k1'] = R_f_one_k1
    result['R_APP'] = R_APP
    result['R_W'] = R_W
    result['R_res'] = R_res
    result['R_T'] = R_T
    result['eta_D'] = eta_D
    result['P_d'] = P_d

    results.append(result)

8010it [00:02, 2944.97it/s]


In [8]:
plot_df = pd.DataFrame(results)


# convert from meters per second to km per hour
ms_to_kmh = 3.6
plot_df['V_s_km'] = plot_df['V_s'] * ms_to_kmh

plot_df.tail()

,T,h_0,V_s,P_installed,R_f,R_f_one_k1,R_APP,R_W,R_res,R_T,eta_D,P_d,V_s_km
8005,2.0,20.0,6.965500,780.0,39.080314,47.982523,4.876663,87.101626,21.080456,161.041269,0.65,1725.743010,25.07580
8006,2.0,20.0,6.974125,780.0,39.170939,48.093792,4.887956,88.275560,21.117007,162.374314,0.65,1742.182715,25.10685
8007,2.0,20.0,6.982750,780.0,39.261662,48.205181,4.899260,89.443398,21.153544,163.701384,0.65,1758.593594,25.13790
8008,2.0,20.0,6.991375,780.0,39.352483,48.316691,4.910577,90.603332,21.190068,165.020668,0.65,1774.955960,25.16895
8009,2.0,20.0,7.000000,780.0,39.443403,48.428321,4.921906,91.753541,21.226580,166.330348,0.65,1791.249900,25.20000


In [ ]:
selected_h_3 = plot_df.query('h_0 == 3 & P_d < P_installed')
selected_h_3_5 = plot_df.query('h_0 == 3.5 & P_d < P_installed')
selected_h_4 = plot_df.query('h_0 == 4 & P_d < P_installed')
selected_h_5 = plot_df.query('h_0 == 5 & P_d < P_installed')
selected_h_6 = plot_df.query('h_0 == 6 & P_d < P_installed')
selected_h_7 = plot_df.query('h_0 == 7 & P_d < P_installed')
selected_h_8 = plot_df.query('h_0 == 8 & P_d < P_installed')
selected_h_9 = plot_df.query('h_0 == 9 & P_d < P_installed')
selected_h_10 = plot_df.query('h_0 == 10 & P_d < P_installed')
selected_h_deep = plot_df.query('h_0 == 20 & P_d < P_installed')
plt.figure(dpi=400)
plt.plot(selected_h_3['V_s_km'],selected_h_3['P_d'], 'pink', linewidth = 1.5)
plt.plot(selected_h_3_5['V_s_km'],selected_h_3_5['P_d'], 'orange', linewidth = 1.5)
plt.plot(selected_h_4['V_s_km'],selected_h_4['P_d'], 'maroon', linewidth = 1.5)
plt.plot(selected_h_5['V_s_km'],selected_h_5['P_d'], 'r', linewidth = 1.5)
# plt.plot(selected_h_6['V_s_km'],selected_h_6['P_d'], 'g', linewidth = 1)
plt.plot(selected_h_7['V_s_km'],selected_h_7['P_d'], 'b', linewidth = 1.5)
# plt.plot(selected_h_8['V_s_km'],selected_h_8['P_d'], 'r', linewidth = 1.0)
# plt.plot(selected_h_9['V_s_km'],selected_h_9['P_d'], 'g', linewidth = 1)
plt.plot(selected_h_10['V_s_km'],selected_h_10['P_d'], 'm', linewidth = 1.5)
plt.plot(selected_h_deep['V_s_km'],selected_h_deep['P_d'], 'k', linewidth = 1.5)

plt.xlabel('V (km/h)')
plt.ylabel('P_D (kW)')
plt.legend(['h=3m','h=3.5m','h=4m','h=5m','h=7m','h=10m','deep water'])
plt.grid()
plt.xlim(0, 21)
plt.ylim(0,800)
plt.title('the relationship between delivered power and vessel speed relative to water of Herso 1 sailing alone \n Holtrop & Mennen_no_correction');
plt.savefig('Holtrop & Mennen_no_correction.pdf',bbox_inches = 'tight', dpi=600, format='pdf')  

In [10]:
eta_D: 0.65